In [1]:
import orjson as json

In [2]:
!ls

__pycache__                    game_mode.ipynb
chat_eda.ipynb                 lobby_type.ipynb
extract_chat_features.ipynb    mappings.py
extract_general_features.ipynb player_logs.ipynb
extract_objectives.ipynb       player_t.ipynb
features.py                    teamfights.ipynb
game_features.ipynb


In [3]:
for line in open('../data/train_matches.jsonl'):
    game = json.loads(line)
    break

In [10]:
for p in game['players']:
    print(p.keys())
    print(game['game_time'], p['times'], p['gold_t'], p['lh_t'], p['xp_t'], p['gold'])
    break

dict_keys(['player_slot', 'hero_id', 'hero_name', 'account_id_hash', 'ability_upgrades', 'obs_placed', 'sen_placed', 'creeps_stacked', 'camps_stacked', 'rune_pickups', 'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'observers_placed', 'stuns', 'max_hero_hit', 'times', 'gold_t', 'lh_t', 'dn_t', 'xp_t', 'obs_log', 'sen_log', 'obs_left_log', 'sen_left_log', 'purchase_log', 'kills_log', 'buyback_log', 'runes_log', 'obs', 'sen', 'actions', 'pings', 'purchase', 'gold_reasons', 'xp_reasons', 'killed', 'item_uses', 'ability_uses', 'hero_hits', 'damage', 'damage_taken', 'damage_inflictor', 'runes', 'killed_by', 'kill_streaks', 'multi_kills', 'life_state', 'healing', 'damage_inflictor_received', 'randomed', 'pred_vict', 'gold', 'lh', 'xp', 'x', 'y', 'hero_inventory', 'hero_stash', 'health', 'max_health', 'max_mana', 'level', 'kills', 'deaths', 'assists', 'denies', 'nearby_creep_death_count'])
155 [0, 60, 120] [1, 91, 356] [0, 0, 4] [0, 107, 387] 543


In [24]:
for p in game['players']:
    print(p['gold_t'], p['gold'])

[1, 91, 356] 543
[1, 91, 262] 399
[1, 91, 182] 304
[1, 169, 260] 389
[1, 91, 281] 402
[1, 331, 738] 982
[1, 251, 561] 788
[1, 251, 342] 531
[1, 251, 342] 796
[1, 425, 680] 851


In [11]:
p = game['players'][0]

In [13]:
times = p['times'] + [game['game_time']]
gold_t = p['gold_t'] + [p['gold']]
xp_t = p['xp_t'] + [p['xp']]

In [22]:
gold_t

[1, 91, 356, 543]

In [14]:
times

[0, 60, 120, 155]

In [15]:
dt = [times[i + 1] - times[i] for i in range(len(times) - 1)]

In [16]:
dt

[60, 60, 35]

In [18]:
dgold = [gold_t[i + 1] - gold_t[i] for i in range(len(p['gold_t']))]

In [20]:
dgold

[90, 265, 187]

In [27]:
gold_sec = [gold / time for gold,time in zip(dgold, dt)]

In [30]:
sum(gold_sec) / len(gold_sec)

3.753174603174603

In [26]:
beta = .6
exp_avg = 0

In [29]:
for gs in gold_sec:
    exp_avg = beta * exp_avg + (1 - beta) * gs
exp_avg

3.413142857142857

In [31]:
corr = exp_avg / (1 - beta ** (len(gold_sec) + 1))

In [32]:
corr

3.9213497899159657

In [71]:
def dt_features(game, player, beta=.6, corrected=True):
    if 'init_dict' not in dir(dt_features):
        dt_features.init_dict = {'xp_sec': 0, 'gold_sec': 0}
    c = dt_features.init_dict.copy()
    
    
    times = player['times'].copy()
    gold_t = player['gold_t'].copy()
    xp_t = player['xp_t'].copy()
    
    if game['game_time'] > times[-1]:
        times.append(game['game_time'])
        gold_t.append(player['gold'])
        xp_t.append(player['xp'])
    n = len(times) - 1
    dt = [times[i + 1] - times[i] for i in range(n)]
    dgold = [gold_t[i + 1] - gold_t[i] for i in range(n)]
    dxp = [xp_t[i + 1] - xp_t[i] for i in range(n)]

    
    gold_sec = [gold / time for gold,time in zip(dgold, dt)]
    avg_gold = 0
    for gs in gold_sec:
        avg_gold = beta * avg_gold + (1 - beta) * gs
    
    xp_sec = [xp / time for xp, time in zip(dxp, dt)]
    avg_xp = 0
    for xs in xp_sec:
        avg_xp = beta * avg_xp + (1 - beta) * xs
    
    if corrected:
        avg_gold = avg_gold / (1 - beta ** (n + 1))
        avg_xp = avg_xp / (1 - beta ** (n + 1))
    
    c['xp_sec'] = avg_xp
    c['gold_sec'] = avg_gold
    return c

In [72]:
for line in open('../data/train_matches.jsonl'):
    game = json.loads(line)
    for p in game['players']:
        print(dt_features(game, p))

{'xp_sec': 3.498818277310925, 'gold_sec': 3.9213497899159657}
{'xp_sec': 3.4923844537815123, 'gold_sec': 2.8328518907563027}
{'xp_sec': 1.0839023109243697, 'gold_sec': 2.2682510504201683}
{'xp_sec': 3.7036502100840334, 'gold_sec': 2.5752363445378146}
{'xp_sec': 2.4157037815126046, 'gold_sec': 2.7100840336134455}
{'xp_sec': 6.406906512605041, 'gold_sec': 5.984112394957982}
{'xp_sec': 5.365414915966387, 'gold_sec': 5.09453781512605}
{'xp_sec': 2.0011817226890756, 'gold_sec': 3.5891544117647056}
{'xp_sec': 2.563681722689075, 'gold_sec': 7.0686712184873945}
{'xp_sec': 5.724789915966387, 'gold_sec': 4.586265756302521}
{'xp_sec': 5.073943708542613, 'gold_sec': 7.246486590191314}
{'xp_sec': 5.068877583364468, 'gold_sec': 5.644800508027843}
{'xp_sec': 8.564625632165173, 'gold_sec': 5.649410716203427}
{'xp_sec': 5.016338602916293, 'gold_sec': 7.5632537075704285}
{'xp_sec': 3.9894131750966246, 'gold_sec': 6.545107383469214}
{'xp_sec': 8.863736666996639, 'gold_sec': 7.760400633236645}
{'xp_sec': 

KeyboardInterrupt: 